# Set up environment and load in data

In [ ]:
library(tidyverse)
library(brms)
library(here)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

data_bc_clean = data_bc_clean %>%
  mutate(correct = ifelse(possiblePayoffleft>possiblePayoffright & leftChosen == 1, 1, ifelse(possiblePayoffleft<possiblePayoffright & leftChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(typeLeft == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoffleft - possiblePayoffright,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-275:-225","-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  filter(!is.na(val_diff_bin_str))

Center variables!

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(day_std = (day - mean(day))/sd(day)) %>%
  group_by(subnum) %>%
  mutate(val_diff_std = (val_diff - mean(val_diff))/sd(val_diff),
         abs_val_diff = abs(val_diff),
         abs_val_diff_std = (abs_val_diff - mean(abs_val_diff))/sd(abs_val_diff))

In [ ]:
data_bc_clean = data_bc_clean %>%
  mutate(day_std = (day - mean(day))/sd(day)) %>%
  group_by(subnum) %>%
  mutate(val_diff_std = (val_diff - mean(val_diff))/sd(val_diff),
         abs_val_diff = abs(val_diff),
         abs_val_diff_std = (abs_val_diff - mean(abs_val_diff))/sd(abs_val_diff))

# Logits for choice in Y/N and BC

## YN task

Does the logit slope change across days? (value_diff:day interaction)  
Is the logit slope different depending on the stim type? (value_diff:type interaction)  
Does it change differently depending on the stim type? (value_diff:day:type interaction)  

Run logits, extract draws and ppc, save output.  

In [ ]:

fn_draws = paste0(helpers_path, '/inputs/yn_logit_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/yn_logit_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  yn_logit_draws = read.csv(fn_draws)
  yn_logit_ppc = read.csv(fn_ppc)
} else{
  subnums = unique(data_yn_clean$subnum)
  
  yn_logit_draws = data_frame()
  yn_logit_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_yn_data = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      select(val_diff_bin_str, val_diff_std, day_std, type, val_diff, day, yesChosen)
    
    
    cur_yn_logit <- brm(formula = yesChosen ~ val_diff_std * day_std * type,
                        data= cur_yn_data,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_yn_logit)
    cur_draws$subnum = cur_sub
    
    yn_logit_draws = rbind(yn_logit_draws, cur_draws)
    
    cur_pred = fitted(cur_yn_logit, newdata = cur_yn_data %>%
                        select(-val_diff, -day, -yesChosen, -val_diff_bin_str), scale = "linear") %>%
      as_tibble() %>%
      transmute(prob = Estimate %>% inv_logit_scaled()) %>%
      bind_cols(cur_yn_data) %>%
      mutate(subnum = cur_sub)
    
    yn_logit_ppc = rbind(yn_logit_ppc, cur_pred)
  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)
  write.csv(yn_logit_ppc, fn_ppc, row.names = F)
  
  rm(cur_yn_data, cur_yn_logit, cur_draws, cur_pred)
}


In [ ]:
# Model diagnostics

## Examine chains for the three variables of interest  

as.data.frame(yn_logit_draws) %>%
  select("b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  ggplot(aes(.iteration, value, color = as.factor(.chain)))+
  geom_line()+
  facet_grid(subnum ~ key, scales = 'free_y')+
  labs(x = "", y = "") +
  theme(legend.position = "none",
        panel.grid = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

## Examine autocorrelation in the variables of interest.  

as.data.frame(yn_logit_draws) %>%
  select("b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  group_by(subnum, .chain, key) %>%
  summarise(lag0 = acf(value, lag.max=1, plot = F)[["acf"]][1],
            lag1 = acf(value, lag.max=1, plot = F)[["acf"]][2],
            lag2 = acf(value, lag.max=2, plot = F)[["acf"]][3],
            lag3 = acf(value, lag.max=3, plot = F)[["acf"]][4],
            lag4 = acf(value, lag.max=4, plot = F)[["acf"]][5],
            lag5 = acf(value, lag.max=5, plot = F)[["acf"]][6],
            lag6 = acf(value, lag.max=6, plot = F)[["acf"]][7],
            lag7 = acf(value, lag.max=7, plot = F)[["acf"]][8],
            lag8 = acf(value, lag.max=8, plot = F)[["acf"]][9],
            lag9 = acf(value, lag.max=9, plot = F)[["acf"]][10],
            lag10 = acf(value, lag.max=10, plot = F)[["acf"]][11],
            lag11 = acf(value, lag.max=11, plot = F)[["acf"]][12], .groups = 'keep') %>%
  gather(lag, cor, -subnum, -.chain, -key) %>%
  mutate(lag = gsub("lag", "", lag),
         lag = as.numeric(lag)) %>%
  ggplot(aes(lag, cor, fill = as.factor(.chain)))+
  geom_bar(stat = "identity", alpha=.5, position = "identity", color = NA)+
  facet_grid(subnum ~ key)+
  labs(x = "Lag", y = "Correlation") +
  theme(legend.position = "none",
        panel.grid = element_blank())+
  scale_x_continuous(breaks = c(0, 5, 10))+
  scale_y_continuous(breaks = c(0, .5, 1))

**Plot predicted plots on top of observed data**

In [ ]:
yn_data_summary = data_yn_clean %>%
  mutate(val_diff_bin_midpoint = round(val_diff/50)*50) %>%
  group_by(subnum, day, val_diff_bin_str, type) %>%
  summarise(sem_yes = sd(yesChosen)/sqrt(n()),
            mean_yes = mean(yesChosen), 
            val_diff = unique(val_diff_bin_midpoint),.groups="keep") %>%
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11")))



p = yn_logit_ppc %>% 
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11"))) %>%
  ggplot()+
  geom_vline(aes(xintercept = 0), color = "light gray")+
  geom_hline(aes(yintercept = 0.5), color = "light gray")+
  geom_line(aes(x = val_diff, y = prob, color=type))+
  geom_point(data = yn_data_summary, aes(x = val_diff, y = mean_yes, color = type)) +
  geom_errorbar(data = yn_data_summary, aes(x = val_diff, y = mean_yes, ymin = mean_yes - sem_yes, ymax = mean_yes + sem_yes, color= type), width = 0) +
  facet_grid(subnum~day) + 
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task - True vs Predicted", color="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,1,.5))

# ggsave(file=paste0(fig_out_path, 'yn_logit_ppc.jpg'), p, height = 6, width = 10, units="in")
p

## BC task

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_logit_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_logit_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_logit_draws = read.csv(fn_draws)
  bc_logit_ppc = read.csv(fn_ppc)
} else{
  subnums = unique(data_bc_clean$subnum)
  
  bc_logit_draws = data_frame()
  bc_logit_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_data = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      select(val_diff_bin_str, val_diff_std, day_std, type, val_diff, day, leftChosen)
    
    
    cur_bc_logit <- brm(formula = leftChosen ~ val_diff_std * day_std * type,
                        data= cur_bc_data,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_logit)
    cur_draws$subnum = cur_sub
    
    bc_logit_draws = rbind(bc_logit_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_logit, newdata = cur_bc_data %>%
                        select(-val_diff, -day, -leftChosen, -val_diff_bin_str), scale = "linear") %>%
      as_tibble() %>%
      transmute(prob = Estimate %>% inv_logit_scaled()) %>%
      bind_cols(cur_bc_data) %>%
      mutate(subnum = cur_sub)
    
    bc_logit_ppc = rbind(bc_logit_ppc, cur_pred)
  }
  names(bc_logit_draws) = gsub(":", ".", names(bc_logit_draws))
  write.csv(bc_logit_draws, fn_draws, row.names = F)
  write.csv(bc_logit_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_data, cur_bc_logit, cur_draws, cur_pred)
}


In [ ]:
# Model diagnostics

## Examine chains for the three variables of interest  

as.data.frame(bc_logit_draws) %>%
  select("b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  ggplot(aes(.iteration, value, color = as.factor(.chain)))+
  geom_line()+
  facet_grid(subnum ~ key, scales = 'free_y')+
  labs(x = "", y = "") +
  theme(legend.position = "none",
        panel.grid = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())


## Examine autocorrelation in the variables of interest.  
as.data.frame(bc_logit_draws) %>%
  select("b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  group_by(subnum, .chain, key) %>%
  summarise(lag0 = acf(value, lag.max=1, plot = F)[["acf"]][1],
            lag1 = acf(value, lag.max=1, plot = F)[["acf"]][2],
            lag2 = acf(value, lag.max=2, plot = F)[["acf"]][3],
            lag3 = acf(value, lag.max=3, plot = F)[["acf"]][4],
            lag4 = acf(value, lag.max=4, plot = F)[["acf"]][5],
            lag5 = acf(value, lag.max=5, plot = F)[["acf"]][6],
            lag6 = acf(value, lag.max=6, plot = F)[["acf"]][7],
            lag7 = acf(value, lag.max=7, plot = F)[["acf"]][8],
            lag8 = acf(value, lag.max=8, plot = F)[["acf"]][9],
            lag9 = acf(value, lag.max=9, plot = F)[["acf"]][10],
            lag10 = acf(value, lag.max=10, plot = F)[["acf"]][11],
            lag11 = acf(value, lag.max=11, plot = F)[["acf"]][12], .groups = 'keep') %>%
  gather(lag, cor, -subnum, -.chain, -key) %>%
  mutate(lag = gsub("lag", "", lag),
         lag = as.numeric(lag)) %>%
  ggplot(aes(lag, cor, fill = as.factor(.chain)))+
  geom_bar(stat = "identity", alpha=.5, position = "identity", color = NA)+
  facet_grid(subnum ~ key)+
  labs(x = "Lag", y = "Correlation") +
  theme(legend.position = "none",
        panel.grid = element_blank())+
  scale_x_continuous(breaks = c(0, 5, 10))+
  scale_y_continuous(breaks = c(0, .5, 1))

**Plot predicted plots on top of observed data**

In [ ]:
bc_data_summary = data_bc_clean %>%
  mutate(val_diff_bin_midpoint = round(val_diff/50)*50) %>%
  group_by(subnum, day, val_diff_bin_str, type) %>%
  summarise(sem_left = sd(leftChosen)/sqrt(n()),
            mean_left = mean(leftChosen), 
            val_diff = unique(val_diff_bin_midpoint),.groups="keep") %>%
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11")))



p = bc_logit_ppc %>% 
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11"))) %>%
  ggplot()+
  geom_vline(aes(xintercept = 0), color = "light gray")+
  geom_hline(aes(yintercept = 0.5), color = "light gray")+
  geom_line(aes(x = val_diff, y = prob, color=type))+
  geom_point(data = bc_data_summary, aes(x = val_diff, y = mean_left, color = type)) +
  geom_errorbar(data = bc_data_summary, aes(x = val_diff, y = mean_left, ymin = mean_left - sem_left, ymax = mean_left + sem_left, color= type), width = 0) +
  facet_grid(subnum~day) + 
  labs(x = "Value Left - Value Right", y = "p(Left)", title="BC Task - True vs Predicted", color="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,1,.5))

# ggsave(file=paste0(fig_out_path, 'bc_logit_ppc.jpg'), p, height = 6, width = 10, units="in")
p

## Parameter distributions

For both tasks parameter distributions collapsed across subjects (similar to a random slopes analysis)

In [ ]:
yn_par_summary = as.data.frame(yn_logit_draws) %>%
  select("b_val_diff_std", "b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum,) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_val_diff_std.day_std", "Value Diff * Day", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", ifelse(key == "b_val_diff_std.day_std.typeRE", "Value Diff * Day * Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", NA)))),
         key = factor(key, levels = c("Value Diff","Value Diff * Type (RE)", "Value Diff * Day", "Value Diff * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

bc_par_summary = as.data.frame(bc_logit_draws) %>%
  select("b_val_diff_std", "b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum,) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_val_diff_std.day_std", "Value Diff * Day", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", ifelse(key == "b_val_diff_std.day_std.typeRE", "Value Diff * Day * Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", NA)))),
         key = factor(key, levels = c("Value Diff", "Value Diff * Type (RE)", "Value Diff * Day", "Value Diff * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

plim = max(abs(bc_par_summary$l95), abs(yn_par_summary$l95) ,bc_par_summary$h95, yn_par_summary$h95)

p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes/Left ~ Value Diff * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

p
# ggsave(file=paste0(fig_out_path, 'ynbc_logit_interactionParEsts.jpg'), p, height = 3, width = 9, units="in")



In [ ]:
as.data.frame(yn_logit_draws) %>%
  select("b_val_diff_std", "b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum,) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_val_diff_std.day_std", "Value Diff * Day", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", ifelse(key == "b_val_diff_std.day_std.typeRE", "Value Diff * Day * Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", NA)))),
         key = factor(key, levels = c("Value Diff","Value Diff * Type (RE)", "Value Diff * Day", "Value Diff * Day * Type (RE)"))) %>%
  group_by(key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) 

In [ ]:
as.data.frame(yn_logit_draws) %>%
  select("b_val_diff_std", "b_val_diff_std.day_std", "b_val_diff_std.typeRE", "b_val_diff_std.day_std.typeRE", subnum,) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_val_diff_std.day_std", "Value Diff * Day", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", ifelse(key == "b_val_diff_std.day_std.typeRE", "Value Diff * Day * Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", NA)))),
         key = factor(key, levels = c("Value Diff","Value Diff * Type (RE)", "Value Diff * Day", "Value Diff * Day * Type (RE)"))) %>%
  group_by(key) %>%
  filter(!subnum %in% c(619, 629)) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) 

## Change in Val Diff Slope

5 slopes per subject: day 1, day 2, day 3, week 2, week 3

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/yn_logit_bydays_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/yn_logit_bydays_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  yn_logit_draws = read.csv(fn_draws)
  yn_logit_ppc = read.csv(fn_ppc)
} else{
  subnums = unique(data_yn_clean$subnum)
  days = list("day 1" = c(1), "day 2" = c(2), "day 3" = c(3), "week 2" = c(4, 5, 6, 7, 8), "week 3" = c(9, 10, 11))
  
  yn_logit_draws = data_frame()
  yn_logit_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    for(j in 1:length(days)){
      cur_sub = subnums[i]
      cur_days = days[[j]]
      
      cur_yn_data = data_yn_clean %>% 
        ungroup() %>%
        filter(subnum == cur_sub & day %in% cur_days) %>%
        select(val_diff_bin_str, val_diff_std, day_std, type, val_diff, day, yesChosen)
      
      
      cur_yn_logit <- brm(formula = yesChosen ~ val_diff_std * type,
                          data= cur_yn_data,
                          family = bernoulli(link = "logit"),
                          warmup = 500,
                          iter = 2500,
                          chains = 4,
                          init = "0",
                          cores = 4,
                          seed = 389253)
      
      cur_draws = as_draws_df(cur_yn_logit)
      cur_draws$subnum = cur_sub
      cur_draws$day = names(days)[j]
      
      yn_logit_draws = rbind(yn_logit_draws, cur_draws)
      
      cur_pred = fitted(cur_yn_logit, newdata = cur_yn_data %>%
                          select(-val_diff, -day, -yesChosen, -val_diff_bin_str), scale = "linear") %>%
        as_tibble() %>%
        transmute(prob = Estimate %>% inv_logit_scaled()) %>%
        bind_cols(cur_yn_data) %>%
        mutate(subnum = cur_sub)
      
      yn_logit_ppc = rbind(yn_logit_ppc, cur_pred)
    }
  }
  names(yn_logit_draws) = gsub(":", ".", names(yn_logit_draws))
  write.csv(yn_logit_draws, fn_draws, row.names = F)
  write.csv(yn_logit_ppc, fn_ppc, row.names = F)
  
  rm(cur_yn_data, cur_yn_logit, cur_draws, cur_pred)
}


In [ ]:

fn_draws = paste0(helpers_path, '/inputs/bc_logit_bydays_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_logit_bydays_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_logit_draws = read.csv(fn_draws)
  bc_logit_ppc = read.csv(fn_ppc)
} else{
  subnums = unique(data_bc_clean$subnum)
  days = list("day 1" = c(1), "day 2" = c(2), "day 3" = c(3), "week 2" = c(4, 5, 6, 7, 8), "week 3" = c(9, 10, 11))
  
  bc_logit_draws = data_frame()
  bc_logit_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    for(j in 1:length(days)){
      cur_sub = subnums[i]
      cur_days = days[[j]]
      
      cur_bc_data = data_bc_clean %>% 
        ungroup() %>%
        filter(subnum == cur_sub & day %in% cur_days) %>%
        select(val_diff_bin_str, val_diff_std, day_std, type, val_diff, day, leftChosen)
      
      
      cur_bc_logit <- brm(formula = leftChosen ~ val_diff_std * type,
                          data= cur_bc_data,
                          family = bernoulli(link = "logit"),
                          warmup = 500,
                          iter = 2500,
                          chains = 4,
                          init = "0",
                          cores = 4,
                          seed = 389253)
      
      cur_draws = as_draws_df(cur_bc_logit)
      cur_draws$subnum = cur_sub
      cur_draws$day = names(days)[j]
      
      bc_logit_draws = rbind(bc_logit_draws, cur_draws)
      
      cur_pred = fitted(cur_bc_logit, newdata = cur_bc_data %>%
                          select(-val_diff, -day, -leftChosen, -val_diff_bin_str), scale = "linear") %>%
        as_tibble() %>%
        transmute(prob = Estimate %>% inv_logit_scaled()) %>%
        bind_cols(cur_bc_data) %>%
        mutate(subnum = cur_sub)
      
      bc_logit_ppc = rbind(bc_logit_ppc, cur_pred)
    }
  }
  names(bc_logit_draws) = gsub(":", ".", names(bc_logit_draws))
  write.csv(bc_logit_draws, fn_draws, row.names = F)
  write.csv(bc_logit_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_data, cur_bc_logit, cur_draws, cur_pred)
}



In [ ]:
yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(subnum, day, b_val_diff_std, b_typeRE, b_val_diff_std.typeRE) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key = ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Value Diff", "Value Diff * Type (RE)"))) %>%
  group_by(subnum, day, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "Y/N")

In [ ]:
as_tibble(yn_logit_draws) %>%
  select(subnum, day, b_val_diff_std, b_typeRE, b_val_diff_std.typeRE) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key = ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Value Diff", "Value Diff * Type (RE)"))) %>%
  # group_by(subnum, day, key) %>%
  filter(day == "day 1") %>%
group_by(day, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "Y/N")

In [ ]:
 as_tibble(yn_logit_draws) %>%
  select(subnum, day, b_val_diff_std, b_typeRE, b_val_diff_std.typeRE) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key = ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Value Diff", "Value Diff * Type (RE)"))) %>%
  filter(day != "day1") %>%
  group_by(key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "Y/N")

In [ ]:
bc_par_summary = as_tibble(bc_logit_draws) %>%
  select(subnum, day, b_val_diff_std, b_typeRE, b_val_diff_std.typeRE) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key = ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_val_diff_std", "Value Diff", ifelse(key == "b_val_diff_std.typeRE", "Value Diff * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Value Diff", "Value Diff * Type (RE)"))) %>%
  group_by(subnum, day, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "BC")

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(day, m))+
  geom_point(position = position_dodge(width=.75), color="blue")+
  geom_errorbar(aes(ymin = l95, ymax = h95), width=.2, position = position_dodge(width=.75), color="blue")+
  facet_grid(key~subnum, scales="free")+
  theme(panel.grid = element_blank(),
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10))+
  labs(x="", y="Parameter Estimate", title="95% HDI for posterior slopes - Y/N Task")

# ggsave(file=paste0(fig_out_path, 'yn_logit_valdiff_slope.jpg'), p, height = 6, width = 9, units="in")
p

In [ ]:
p = bc_par_summary %>%
  ggplot(aes(day, m))+
  geom_point(position = position_dodge(width=.75), color="red")+
  geom_errorbar(aes(ymin = l95, ymax = h95), width=.2, position = position_dodge(width=.75), color="red")+
  facet_grid(key~subnum, scales="free")+
  theme(panel.grid = element_blank(),
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10))+
  labs(x="", y="Parameter Estimate", title="95% HDI for posterior slopes - BC Task")

# ggsave(file=paste0(fig_out_path, 'bc_logit_valdiff_slope.jpg'), p, height = 6, width = 9, units="in")
p

In [ ]:
p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(day, m, color=task, alpha=task))+
  geom_point(position = position_dodge(width=.75))+
  geom_errorbar(aes(ymin = l95, ymax = h95), width=.2, position = position_dodge(width=.75))+
  facet_grid(key~subnum, scales="free")+
  theme(panel.grid = element_blank(),
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10))+
  labs(x="", y="Parameter Estimate", title="95% HDI for posterior slopes", color="")+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(0, 1))+
  guides(alpha="none")

# ggsave(file=paste0(fig_out_path, 'ynbc_logit_valdiff_slope.jpg'), p, height = 6, width = 9, units="in")
p

# RT slope 

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(log_rt = log(rt))

data_bc_clean = data_bc_clean %>%
  mutate(log_rt = log(rt))

## YN task

In [ ]:

fn_draws = paste0(helpers_path, '/inputs/yn_rt_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/yn_rt_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  yn_rt_draws = read.csv(fn_draws)
  yn_rt_ppc = read.csv(fn_ppc)
} else{
  subnums = unique(data_yn_clean$subnum)
  
  yn_rt_draws = data_frame()
  yn_rt_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_yn_data = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      select(abs_val_diff_std, day_std, type, abs_val_diff, day, log_rt)
    
    
    cur_yn_rt <- brm(formula = log_rt ~ abs_val_diff_std * day_std * type,
                     data= cur_yn_data,
                     warmup = 500,
                     iter = 2500,
                     chains = 4,
                     init = "0",
                     cores = 4,
                     seed = 389253)
    
    cur_draws = as_draws_df(cur_yn_rt)
    cur_draws$subnum = cur_sub
    
    yn_rt_draws = rbind(yn_rt_draws, cur_draws)
    
    cur_pred = fitted(cur_yn_rt, newdata = cur_yn_data %>%
                        select(-abs_val_diff, -day, -log_rt)) %>% 
      as_tibble() %>%
      transmute(pred_log_rt = Estimate) %>% #same as mean brms::posterior_epred values not brms::posterior_predict
      bind_cols(cur_yn_data) %>%
      mutate(subnum = cur_sub)
    
    yn_rt_ppc = rbind(yn_rt_ppc, cur_pred)
  }
  names(yn_rt_draws) = gsub(":", ".", names(yn_rt_draws))
  write.csv(yn_rt_draws, fn_draws, row.names = F)
  write.csv(yn_rt_ppc, fn_ppc, row.names = F)
  
  rm(cur_yn_data, cur_yn_rt, cur_draws, cur_pred)
}


In [ ]:
# Model diagnostics

## Examine chains for the three variables of interest  

as.data.frame(yn_rt_draws) %>%
  select("b_abs_val_diff_std.day_std", "b_abs_val_diff_std.typeRE", "b_abs_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  ggplot(aes(.iteration, value, color = as.factor(.chain)))+
  geom_line()+
  facet_grid(subnum ~ key, scales = 'free_y')+
  labs(x = "", y = "") +
  theme(legend.position = "none",
        panel.grid = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

## Examine autocorrelation in the variables of interest.  

as.data.frame(yn_rt_draws) %>%
  select("b_abs_val_diff_std.day_std", "b_abs_val_diff_std.typeRE", "b_abs_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  group_by(subnum, .chain, key) %>%
  summarise(lag0 = acf(value, lag.max=1, plot = F)[["acf"]][1],
            lag1 = acf(value, lag.max=1, plot = F)[["acf"]][2],
            lag2 = acf(value, lag.max=2, plot = F)[["acf"]][3],
            lag3 = acf(value, lag.max=3, plot = F)[["acf"]][4],
            lag4 = acf(value, lag.max=4, plot = F)[["acf"]][5],
            lag5 = acf(value, lag.max=5, plot = F)[["acf"]][6],
            lag6 = acf(value, lag.max=6, plot = F)[["acf"]][7],
            lag7 = acf(value, lag.max=7, plot = F)[["acf"]][8],
            lag8 = acf(value, lag.max=8, plot = F)[["acf"]][9],
            lag9 = acf(value, lag.max=9, plot = F)[["acf"]][10],
            lag10 = acf(value, lag.max=10, plot = F)[["acf"]][11],
            lag11 = acf(value, lag.max=11, plot = F)[["acf"]][12], .groups = 'keep') %>%
  gather(lag, cor, -subnum, -.chain, -key) %>%
  mutate(lag = gsub("lag", "", lag),
         lag = as.numeric(lag)) %>%
  ggplot(aes(lag, cor, fill = as.factor(.chain)))+
  geom_bar(stat = "identity", alpha=.5, position = "identity", color = NA)+
  facet_grid(subnum ~ key)+
  labs(x = "Lag", y = "Correlation") +
  theme(legend.position = "none",
        panel.grid = element_blank())+
  scale_x_continuous(breaks = c(0, 5, 10))+
  scale_y_continuous(breaks = c(0, .5, 1))

Plot log rt over absolute value difference to compare to posterior predictive data

In [ ]:
p = data_yn_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  group_by(subnum, week, abs_val_diff_bin_str, type) %>%
  summarise(mean_log_rt = mean(log(rt)),
            .groups="keep") %>%
  ungroup() %>%
  group_by(week, abs_val_diff_bin_str, type) %>%
  summarise(sem_log_rt = sd(mean_log_rt)/sqrt(n()),
            mean_log_rt = mean(mean_log_rt), .groups = "keep") %>%
  ggplot(aes(abs_val_diff_bin_str, mean_log_rt, color=type))+
  geom_point(size = 2.5)+
  geom_errorbar(aes(ymin=mean_log_rt-sem_log_rt, ymax=mean_log_rt+sem_log_rt), width=0, size = 1)+
  geom_line(aes(group=type))+
  facet_grid(.~week)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task", color="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")+
  # scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(-1,.5,.5))

p
# ggsave(file=paste0(fig_out_path, 'yn_group_RTOverAbsValDiff.jpg'), p, height = 3, width=9, units="in")

Plot predicted over true

In [ ]:
yn_data_summary = data_yn_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")),
         abs_val_diff_bin_midpoint = round(abs(val_diff)/50)*50) %>%
  group_by(subnum, day, abs_val_diff_bin_str, type) %>%
  summarise(mean_log_rt = mean(log(rt)),
            sem_log_rt = sd(log(rt))/sqrt(n()),
            abs_val_diff = unique(abs_val_diff_bin_midpoint),
            .groups="keep") %>%
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11")))

p = yn_rt_ppc %>% 
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11"))) %>%
  ggplot()+
  geom_line(aes(x = abs_val_diff, y = pred_log_rt, color=type))+
  geom_point(data = yn_data_summary, aes(x = abs_val_diff, y = mean_log_rt, color = type)) +
  geom_errorbar(data = yn_data_summary, aes(x = abs_val_diff, y = mean_log_rt, ymin = mean_log_rt - sem_log_rt, ymax = mean_log_rt + sem_log_rt, color= type), width = 0) +
  facet_grid(subnum~day) + 
  labs(x = "|Value Stim - Value Reference|", y = "Log RT", title="Y/N Task - True vs Predicted", color="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")

# ggsave(file=paste0(fig_out_path, 'yn_rt_ppc.jpg'), p, height = 6, width = 10, units="in")
p

## BC task

In [ ]:

fn_draws = paste0(helpers_path, '/inputs/bc_rt_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_rt_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_rt_draws = read.csv(fn_draws)
  bc_rt_ppc = read.csv(fn_ppc)
} else{
  subnums = unique(data_bc_clean$subnum)
  
  bc_rt_draws = data_frame()
  bc_rt_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_data = data_bc_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      select(abs_val_diff_std, day_std, type, abs_val_diff, day, log_rt)
    
    
    cur_bc_rt <- brm(formula = log_rt ~ abs_val_diff_std * day_std * type,
                     data= cur_bc_data,
                     warmup = 500,
                     iter = 2500,
                     chains = 4,
                     init = "0",
                     cores = 4,
                     seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_rt)
    cur_draws$subnum = cur_sub
    
    bc_rt_draws = rbind(bc_rt_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_rt, newdata = cur_bc_data %>%
                        select(-abs_val_diff, -day, -log_rt)) %>% 
      as_tibble() %>%
      transmute(pred_log_rt = Estimate) %>% #same as mean brms::posterior_epred values not brms::posterior_predict
      bind_cols(cur_bc_data) %>%
      mutate(subnum = cur_sub)
    
    bc_rt_ppc = rbind(bc_rt_ppc, cur_pred)
  }
  names(bc_rt_draws) = gsub(":", ".", names(bc_rt_draws))
  write.csv(bc_rt_draws, fn_draws, row.names = F)
  write.csv(bc_rt_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_data, cur_bc_rt, cur_draws, cur_pred)
}


In [ ]:
# Model diagnostics

## Examine chains for the three variables of interest  

as.data.frame(bc_rt_draws) %>%
  select("b_abs_val_diff_std.day_std", "b_abs_val_diff_std.typeRE", "b_abs_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  ggplot(aes(.iteration, value, color = as.factor(.chain)))+
  geom_line()+
  facet_grid(subnum ~ key, scales = 'free_y')+
  labs(x = "", y = "") +
  theme(legend.position = "none",
        panel.grid = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

## Examine autocorrelation in the variables of interest.  

as.data.frame(bc_rt_draws) %>%
  select("b_abs_val_diff_std.day_std", "b_abs_val_diff_std.typeRE", "b_abs_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  group_by(subnum, .chain, key) %>%
  summarise(lag0 = acf(value, lag.max=1, plot = F)[["acf"]][1],
            lag1 = acf(value, lag.max=1, plot = F)[["acf"]][2],
            lag2 = acf(value, lag.max=2, plot = F)[["acf"]][3],
            lag3 = acf(value, lag.max=3, plot = F)[["acf"]][4],
            lag4 = acf(value, lag.max=4, plot = F)[["acf"]][5],
            lag5 = acf(value, lag.max=5, plot = F)[["acf"]][6],
            lag6 = acf(value, lag.max=6, plot = F)[["acf"]][7],
            lag7 = acf(value, lag.max=7, plot = F)[["acf"]][8],
            lag8 = acf(value, lag.max=8, plot = F)[["acf"]][9],
            lag9 = acf(value, lag.max=9, plot = F)[["acf"]][10],
            lag10 = acf(value, lag.max=10, plot = F)[["acf"]][11],
            lag11 = acf(value, lag.max=11, plot = F)[["acf"]][12], .groups = 'keep') %>%
  gather(lag, cor, -subnum, -.chain, -key) %>%
  mutate(lag = gsub("lag", "", lag),
         lag = as.numeric(lag)) %>%
  ggplot(aes(lag, cor, fill = as.factor(.chain)))+
  geom_bar(stat = "identity", alpha=.5, position = "identity", color = NA)+
  facet_grid(subnum ~ key)+
  labs(x = "Lag", y = "Correlation") +
  theme(legend.position = "none",
        panel.grid = element_blank())+
  scale_x_continuous(breaks = c(0, 5, 10))+
  scale_y_continuous(breaks = c(0, .5, 1))

Plot log rt over absolute value difference to compare to posterior predictive data

In [ ]:
p = data_bc_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  group_by(subnum, week, abs_val_diff_bin_str, type) %>%
  summarise(mean_log_rt = mean(log(rt)),
            .groups="keep") %>%
  ungroup() %>%
  group_by(week, abs_val_diff_bin_str, type) %>%
  summarise(sem_log_rt = sd(mean_log_rt)/sqrt(n()),
            mean_log_rt = mean(mean_log_rt), .groups = "keep") %>%
  ggplot(aes(abs_val_diff_bin_str, mean_log_rt, color=type))+
  geom_point(size = 2.5)+
  geom_errorbar(aes(ymin=mean_log_rt-sem_log_rt, ymax=mean_log_rt+sem_log_rt), width=0, size = 1)+
  geom_line(aes(group=type))+
  facet_grid(.~week)+
  labs(x = "|Value Left - Value Right|", y = "Mean Log RT", title="BC Task", color="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")+
  # scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(-1,.5,.5))

p
# ggsave(file=paste0(fig_out_path, 'bc_group_RTOverAbsValDiff.jpg'), p, height = 3, width=9, units="in")

Plot predicted over true

In [ ]:
bc_data_summary = data_bc_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")),
         abs_val_diff_bin_midpoint = round(abs(val_diff)/50)*50) %>%
  group_by(subnum, day, abs_val_diff_bin_str, type) %>%
  summarise(mean_log_rt = mean(log(rt)),
            sem_log_rt = sd(log(rt))/sqrt(n()),
            abs_val_diff = unique(abs_val_diff_bin_midpoint),
            .groups="keep") %>%
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11")))

p = bc_rt_ppc %>% 
  filter(day %in% c(1, 3, 7, 11)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 1", "Day 3", "Day 7", "Day 11"))) %>%
  ggplot()+
  geom_line(aes(x = abs_val_diff, y = pred_log_rt, color=type))+
  geom_point(data = bc_data_summary, aes(x = abs_val_diff, y = mean_log_rt, color = type)) +
  geom_errorbar(data = bc_data_summary, aes(x = abs_val_diff, y = mean_log_rt, ymin = mean_log_rt - sem_log_rt, ymax = mean_log_rt + sem_log_rt, color= type), width = 0) +
  facet_grid(subnum~day) + 
  labs(x = "|Value Left - Value Right|", y = "Log RT", title="BC Task - True vs Predicted", color="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")

# ggsave(file=paste0(fig_out_path, 'bc_rt_ppc.jpg'), p, height = 6, width = 10, units="in")
p

Plot regression parameter estimates for both tasks in one plot

In [ ]:
yn_par_summary = as.data.frame(yn_rt_draws) %>%
  select("b_abs_val_diff_std" ,"b_abs_val_diff_std.day_std", "b_abs_val_diff_std.typeRE", "b_abs_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  mutate(key = ifelse(key == "b_abs_val_diff_std.day_std", "|Value Diff| * Day", ifelse(key == "b_abs_val_diff_std.typeRE", "|Value Diff| * Type (RE)", ifelse(key == "b_abs_val_diff_std.day_std.typeRE", "|Value Diff| * Day * Type (RE)", ifelse(key == "b_abs_val_diff_std", "|Value Diff|", NA)))),
         key = factor(key, levels = c("|Value Diff|","|Value Diff| * Type (RE)", "|Value Diff| * Day", "|Value Diff| * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

In [ ]:
as.data.frame(yn_rt_draws) %>%
  select("b_abs_val_diff_std" ,"b_abs_val_diff_std.day_std", "b_abs_val_diff_std.typeRE", "b_abs_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  mutate(key = ifelse(key == "b_abs_val_diff_std.day_std", "|Value Diff| * Day", ifelse(key == "b_abs_val_diff_std.typeRE", "|Value Diff| * Type (RE)", ifelse(key == "b_abs_val_diff_std.day_std.typeRE", "|Value Diff| * Day * Type (RE)", ifelse(key == "b_abs_val_diff_std", "|Value Diff|", NA)))),
         key = factor(key, levels = c("|Value Diff|","|Value Diff| * Type (RE)", "|Value Diff| * Day", "|Value Diff| * Day * Type (RE)"))) %>%
  group_by(key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) 

In [ ]:
bc_par_summary = as.data.frame(bc_rt_draws) %>%
  select("b_abs_val_diff_std" ,"b_abs_val_diff_std.day_std", "b_abs_val_diff_std.typeRE", "b_abs_val_diff_std.day_std.typeRE", subnum, .chain, .iteration) %>%
  gather(key, value, -subnum, -.chain, -.iteration) %>%
  mutate(key = ifelse(key == "b_abs_val_diff_std.day_std", "|Value Diff| * Day", ifelse(key == "b_abs_val_diff_std.typeRE", "|Value Diff| * Type (RE)", ifelse(key == "b_abs_val_diff_std.day_std.typeRE", "|Value Diff| * Day * Type (RE)", ifelse(key == "b_abs_val_diff_std", "|Value Diff|", NA)))),
         key = factor(key, levels = c("|Value Diff|","|Value Diff| * Type (RE)", "|Value Diff| * Day", "|Value Diff| * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC")

## Parameter distributions
 
For both tasks parameter distributions collapsed across subjects (similar to a random slopes analysis)

In [ ]:
plim = max(abs(bc_par_summary$l95), abs(yn_par_summary$l95) ,bc_par_summary$h95, yn_par_summary$h95)

p = yn_par_summary %>%
  rbind(bc_par_summary) %>%
  ggplot(aes(m, subnum, color=task,  alpha=task)) +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - RT ~ |Value Diff| * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))+
  scale_color_manual(values = c("red", "blue"))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha="none")

p
# ggsave(file=paste0(fig_out_path, 'ynbc_rt_interactionParEsts.jpg'), p, height = 3, width = 9, units="in")



## Bimodality

Is there evidence for some bi-modality for the repeated stimuli (especially after learning), which would suggest that choice is made with a “different fast process” with some probability.

For most subjects HTs are faster than RE for all value bins and almost every day but the difference does not change across days with learning.

In [ ]:
subjs = unique(data_yn_clean$subnum)

for(i in 1:length(subjs)){
  cur_sub = subjs[i]
  
  p = data_yn_clean %>%
    filter(subnum == cur_sub) %>%
    filter(abs_val_diff<175) %>%
    filter(rt < 3) %>%
    group_by(type, day, val_diff_bin_str) %>%
    summarise(mean_log_rt = mean(log(rt)),
              sem_log_rt = sd(log(rt))/sqrt(n()),
              .groups = "keep") %>%
    mutate(day = factor(day, levels = seq(1, 11, 1))) %>%
    ggplot(aes(day, mean_log_rt, color=type))+
    geom_point(position = position_dodge(width=.9))+
    geom_errorbar(aes(ymin = mean_log_rt - sem_log_rt, ymax = mean_log_rt + sem_log_rt, alpha = .5), position = position_dodge(width=.9), width=0)+
    facet_grid(.~ val_diff_bin_str) + 
    scale_color_brewer(palette = "Dark2")+
    theme(panel.grid = element_blank(),
          legend.position = "none",
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          legend.box.margin=margin(t = -10))+
    scale_x_discrete(breaks = c(1, 3, 5, 7, 9, 11))+
    labs(y="Mean Log RT", x = "Day", fill="", title = paste0("Y/N Task - ", cur_sub))
  
  ggsave(file=paste0(fig_out_path, 'yn_',cur_sub,'_logRTSummary.jpg'), p, height = 5, width = 8, units="in")
  
}



In [ ]:

subjs = unique(data_yn_clean$subnum)

for(i in 1:length(subjs)){
  cur_sub = subjs[i]
  
  tmp = data_yn_clean %>%
    filter(subnum == cur_sub) %>%
    filter(abs_val_diff<175) %>%
    filter(rt < 3) %>%
    group_by(type, day, val_diff_bin_str) %>%
    summarise(mean_log_rt = mean(log(rt)),
              sem_log_rt = sd(log(rt))/sqrt(n()),
              .groups = "keep")
  
  tmp1 = tmp %>% filter(type == "HT") %>% 
    rename(mean_log_rt_HT = mean_log_rt, sem_log_rt_HT = sem_log_rt) %>%
    ungroup() %>%
    select(-type)
  
  tmp2 = tmp %>% filter(type == "RE") %>% 
    rename(mean_log_rt_RE = mean_log_rt, sem_log_rt_RE = sem_log_rt) %>%
    ungroup() %>%
    select(-type)
  
  tmp = tmp1 %>%
    left_join(tmp2, by=c("day", "val_diff_bin_str"))
  
  p = tmp %>%
    mutate(mean_log_rt_diff = mean_log_rt_HT - mean_log_rt_RE,
           sem_log_rt_diff = sem_log_rt_HT + sem_log_rt_RE) %>%
    mutate(day = factor(day, levels = seq(1, 11, 1))) %>%
    ggplot(aes(day, mean_log_rt_diff))+
    geom_point(position = position_dodge(width=.9))+
    geom_errorbar(aes(ymin = mean_log_rt_diff - sem_log_rt_diff, ymax = mean_log_rt_diff + sem_log_rt_diff, alpha = .5), position = position_dodge(width=.9), width=0)+
    facet_grid(.~ val_diff_bin_str) + 
    theme(panel.grid = element_blank(),
          legend.position = "none",
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          legend.box.margin=margin(t = -10))+
    scale_x_discrete(breaks = c(1, 3, 5, 7, 9, 11))+
    labs(y="Mean Log RT DIff", x = "Day", fill="", title = paste0("Y/N Task - ", cur_sub))
  
  ggsave(file=paste0(fig_out_path, 'yn_',cur_sub,'_logRTDiff.jpg'), p, height = 5, width = 8, units="in")
  
}

rm(tmp, tmp1, tmp2)

In [ ]:
subjs = unique(data_yn_clean$subnum)

for(i in 1:length(subjs)){
  cur_sub = subjs[i]
  
  p = data_yn_clean %>%
    filter(subnum == cur_sub) %>%
    filter(abs_val_diff<175) %>%
    filter(rt < 3) %>%
    ggplot(aes(rt, fill=type))+
    geom_density(alpha=.5, color=NA)+
    # facet_grid(val_diff_bin_str ~ day, scales = "free_y")+
    facet_grid(day ~ val_diff_bin_str, scales = "free_y")+
    scale_fill_brewer(palette = "Dark2")+
    theme(panel.grid = element_blank(),
          legend.position = "none",
          axis.text = element_blank(),
          axis.ticks = element_blank(),
          legend.box.margin=margin(t = -10))+
    labs(y="", x = "RT", fill="", title = paste0("Y/N Task - ", cur_sub))
  
  ggsave(file=paste0(fig_out_path, 'yn_',cur_sub,'_fullRTdists.jpg'), p, height = 5, width = 8, units="in")
  
}



In [ ]:
subjs = unique(data_bc_clean$subnum)

for(i in 1:length(subjs)){
  cur_sub = subjs[i]
  
  p = data_bc_clean %>%
    filter(subnum == cur_sub) %>%
    filter(abs_val_diff<175) %>%
    filter(rt < 3) %>%
    ggplot(aes(rt, fill=type))+
    geom_density(alpha=.5, color=NA)+
    # facet_grid(val_diff_bin_str ~ day, scales = "free_y")+
    facet_grid(day ~ val_diff_bin_str, scales = "free_y")+
    scale_fill_brewer(palette = "Dark2")+
    theme(panel.grid = element_blank(),
          legend.position = "none",
          axis.text = element_blank(),
          axis.ticks = element_blank(),
          legend.box.margin=margin(t = -10))+
    labs(y="", x = "RT", fill="", title = paste0("BC Task - ", cur_sub))
  
  ggsave(file=paste0(fig_out_path, 'bc_',cur_sub,'_fullRTdists.jpg'), p, height = 5, width = 8, units="in")
  
}


# DDM for YN

In [ ]:
stim_types = c("HT", "RE")
days = as.character(seq(1,11,1))

in_path = file.path(here(), "inputs")

yn_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(days)){
    
    cur_stim_type = stim_types[i]
    cur_day = days[j]
    
    if(file.exists(file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_day_', cur_day,'.csv')))){
      cur_summary = read.csv(file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_day_', cur_day,'.csv')))
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               day = cur_day)
      
      yn_ddm_summary = rbind(yn_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}

In [ ]:
p = yn_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, day, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name))),
         par_group = ifelse(par_group == 'p[1]', "601", ifelse(par_group == "p[2]", "609", ifelse(par_group == 'p[3]', "611", ifelse(par_group == "p[4]", "619", ifelse(par_group == "p[5]", "621", ifelse(par_group == "p[6]", "629", par_group)))))),
         day = as.numeric(day)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt"))) %>%
  ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.75))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.75), width=.5)+
  facet_grid(par_name ~ par_group, scale = "free_y")+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "Parameter Estimate", color="", title = "YN HDDM Estimates (95% HDI)")+
  scale_x_continuous(breaks = seq(1,11,1),
                     labels = c("1", "", "3", "", "5", "", "7", "", "9", "", "11"))

p
# ggsave(file=paste0(fig_out_path, 'yn_subj_hddm_parests_build.jpg'), p, height = 6, width=9, units="in")

In [ ]:
p = yn_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, day, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", par_name),
         day = as.numeric(day)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group == "mu" ) %>%
  
  ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.9))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.9), width=.5)+
  facet_wrap(~ par_name, scales = "free_y", ncol = 5)+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "", color="",  title = "YN HDDM Group Estimates (95% HDI)")+
  scale_x_continuous(breaks = seq(1,11,1),
                     labels = c("1", "", "3", "", "5", "", "7", "", "9", "", "11"))

p
# ggsave(file=paste0(fig_out_path, 'yn_group_hddm_parests.jpg'), p, height = 3, width=9, units="in")

# DDM for BC

In [ ]:
stim_types = c("HT", "RE")
days = as.character(seq(1,11,1))

in_path = file.path(here(), "inputs")

bc_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(days)){
    
    cur_stim_type = stim_types[i]
    cur_day = days[j]
    
    if(file.exists(file.path(in_path, paste0('summary_BC_HDDM_FIT_', cur_stim_type,'_day_', cur_day,'.csv')))){
      cur_summary = read.csv(file.path(in_path, paste0('summary_BC_HDDM_FIT_', cur_stim_type,'_day_', cur_day,'.csv')))
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               day = cur_day)
      
      bc_ddm_summary = rbind(bc_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}

In [ ]:
p = bc_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, day, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
    mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name))),
         par_group = ifelse(par_group == 'p[1]', "601", ifelse(par_group == "p[2]", "609", ifelse(par_group == 'p[3]', "611", ifelse(par_group == "p[4]", "619", ifelse(par_group == "p[5]", "621", ifelse(par_group == "p[6]", "629", par_group)))))),
         day = as.numeric(day)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt"))) %>%
 ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.75))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.75), width=.5)+
  facet_grid(par_name ~ par_group, scale = "free_y")+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "Parameter Estimate", color="", title = "BC HDDM Estimates (95% HDI) - non-scan Days")+
  scale_x_continuous(breaks = seq(1,11,1),
                     labels = c("1", "", "3", "", "5", "", "7", "", "9", "", "11"))

p
# ggsave(file=paste0(fig_out_path, 'bc_subj_hddm_parests.jpg'), p, height = 6, width=9, units="in")

In [ ]:
p = bc_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, day, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", par_name),
         day = as.numeric(day)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group == "mu" ) %>%
  ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.9))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.9), width=.5)+
  facet_wrap(~ par_name, scales = "free_y", ncol = 5)+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "", color="",  title = "BC HDDM Group Estimates (95% HDI) - non-scan Days")+
  scale_x_continuous(breaks = seq(1,11,1),
                     labels = c("1", "", "3", "", "5", "", "7", "", "9", "", "11"))

p
# ggsave(file=paste0(fig_out_path, 'bc_group_hddm_parests.jpg'), p, height = 3, width=9, units="in")

# aDDM for BC

5 rows for pars
3 columns for each day

Each plot: x is value with vertical line on 0, y is parameter name with group distribution on top and subject distributions under

In [ ]:
stim_types = c("HT", "RE")
days = c("3", "7", "11")

in_path = file.path(here(), "inputs")

bc_addm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(days)){
    cur_stim_type = stim_types[i]
    cur_day = days[j]
    cur_summary = read.csv(file.path(in_path, paste0('summary_BC_HaDDM_FIT_', cur_stim_type,'_day_', cur_day,'.csv')))
    cur_summary = cur_summary %>%
      rename(par = X) %>%
      mutate(stim_type = cur_stim_type,
             day = cur_day)
    
    bc_addm_summary = rbind(bc_addm_summary, cur_summary)
  }
}

In [ ]:
p = bc_addm_summary %>%
  select(par, Lower95, Upper95, Mean, day, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", par_name),
         day = factor(day, levels = c("3", "7", "11"))) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.75))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.75), width=.5)+
  facet_grid(par_name ~ par_group, scale = "free_y")+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "Parameter Estimate", color="", title = "B/C HaDDM Estimates (95% HDI)")

p
# ggsave(file=paste0(fig_out_path, 'bc_subj_haddm_parests.jpg'), p, height = 6, width=9, units="in")

In [ ]:
p = bc_addm_summary %>%
  select(par, Lower95, Upper95, Mean, day, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", par_name),
         day = as.numeric(day)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group == "mu" ) %>%
  
  ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.9))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.9), width=.5)+
  facet_wrap(~ par_name, scales = "free_y", ncol = 5)+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "", color="",  title = "B/C HaDDM Group Estimates (95% HDI)")+
  scale_x_continuous(breaks = c(3, 7, 11))

p
# ggsave(file=paste0(fig_out_path, 'bc_group_haddm_parests.jpg'), p, height = 3, width=9, units="in")